In [69]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
from statsmodels.graphics.mosaicplot import mosaic
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold

In [70]:
df = pd.read_csv("../dataset_diabetes/diabetic_data.csv")

In [87]:
knn_data = df.drop(['encounter_id','patient_nbr', 'weight', 'examide', 'citoglipton','diag_1','diag_2','diag_3'],axis=1)
#change ? and None to Nan
data_clean = knn_data.replace('?', np.nan)
data_clean = data_clean.replace('None', np.nan)
data_clean = data_clean.drop(['payer_code','medical_specialty','max_glu_serum','A1Cresult'],axis=1)
#numerical columns: Time in hospital,Number of lab procedures,Number of procedures,
#Number of medications,Number of outpatient visits,Number of emergency visits,
#Number of inpatient visits,Number of diagnoses,

numerical = ['time_in_hospital','num_lab_procedures','num_procedures','num_medications',
             'number_outpatient','number_emergency','number_inpatient',
             'number_diagnoses','admission_type_id','discharge_disposition_id','admission_source_id']

numerical_df = data_clean[numerical]
#want just the Y's
output_var = data_clean['readmitted']  
non_numerical_df = data_clean.drop(numerical,axis=1)
non_numerical_df = non_numerical_df.drop(['readmitted'],axis=1)
df1 = pd.DataFrame()
df2 = pd.DataFrame()
for i in non_numerical_df.columns:
    if (len(set(non_numerical_df[i])) > 2): #if set is greater than 2, want to get a separate df
        df1[i] = non_numerical_df[i]
    else: #if set is less than 2, want another df
        df2[i] = non_numerical_df[i]
dummy2 = pd.get_dummies(df2) #change to dummy variables
dummy1 = pd.get_dummies(df1)
frames = [numerical_df,dummy1,dummy2,output_var]
cleaned_df = pd.concat(frames,axis=1)

cleaned_df['readmitted'] = cleaned_df['readmitted'].map( {'NO': 0, '>30': 1, '<30': 1} ).astype(int)

# Create a new column that for each row, generates a random number between 0 and 1, and
# if that value is less than or equal to .75, then sets the value of that cell as True
# and false otherwise. This is a quick and dirty way of randomly assigning some rows to
# be used as the training data and some as the test data.
cleaned_df['is_train'] = np.random.uniform(0, 1, len(cleaned_df)) <= .75
# Create two new dataframes, one with the training rows, one with the test rows
train, test = cleaned_df[cleaned_df['is_train']==True], cleaned_df[cleaned_df['is_train']==False]
# Create a list of the feature column's names
features = cleaned_df.columns[:-2]
#y = pd.factorize(train['readmitted'])[0]
y = train['readmitted']

In [88]:
train_clean = train.drop(['is_train'],axis=1)
test_clean = test.drop(['is_train'],axis=1)

In [89]:
train_clean.values

array([[ 1, 41,  0, ...,  1,  0,  0],
       [ 2, 11,  5, ...,  0,  1,  0],
       [ 2, 44,  1, ...,  0,  1,  0],
       ..., 
       [ 5, 33,  3, ...,  0,  1,  0],
       [ 1, 53,  0, ...,  0,  1,  0],
       [ 6, 13,  3, ...,  1,  0,  0]])

In [90]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train) #fit

    def predict(self, x):
        return self.clf.predict(x) #predict
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [91]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return(oof_train.reshape(-1, 1), oof_test.reshape(-1, 1))

In [92]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [93]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [94]:
train.head(3)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,admission_type_id,discharge_disposition_id,...,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted,is_train
0,1,41,0,1,0,0,0,1,6,25,...,1,0,1,0,0,1,1,0,0,True
2,2,11,5,13,2,0,1,6,1,1,...,1,0,1,0,0,1,0,1,0,True
3,2,44,1,16,0,0,0,7,1,1,...,1,0,1,0,1,0,0,1,0,True


In [96]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['readmitted'].ravel()
features = cleaned_df.columns[:-2]
train = train_clean[features]
test = test_clean[features]
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data

In [97]:
x_train

array([[ 1, 41,  0, ...,  1,  1,  0],
       [ 2, 11,  5, ...,  1,  0,  1],
       [ 2, 44,  1, ...,  0,  0,  1],
       ..., 
       [ 5, 33,  3, ...,  1,  0,  1],
       [ 1, 53,  0, ...,  0,  0,  1],
       [ 6, 13,  3, ...,  1,  1,  0]])

In [98]:
x_test

array([[ 3, 59,  0, ...,  0,  0,  1],
       [ 1, 51,  0, ...,  0,  0,  1],
       [ 9, 47,  2, ...,  1,  0,  1],
       ..., 
       [ 2, 46,  6, ...,  1,  0,  1],
       [ 1,  1,  0, ...,  0,  0,  1],
       [10, 45,  2, ...,  0,  0,  1]])

In [99]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [101]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

Training is complete


In [102]:
rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train,y_train)

[  1.82492709e-02   1.74537425e-02   1.46717675e-02   3.84788633e-02
   9.86989157e-02   1.30347400e-01   3.23392804e-01   9.90428323e-02
   1.14785667e-02   8.10544292e-02   3.45719827e-02   2.36606906e-03
   7.84745151e-04   9.60120743e-03   1.39975021e-03   9.21511831e-04
   1.73008250e-03   1.55674870e-03   0.00000000e+00   1.81655396e-04
   3.54847104e-04   1.40601368e-03   2.37748193e-03   2.02960373e-03
   2.36695462e-03   9.02664948e-04   3.38807814e-03   4.97229944e-03
   2.34640986e-03   3.57723083e-04   8.38641238e-03   4.53421363e-03
   5.35499003e-04   3.47021655e-05   1.47972941e-03   1.28645941e-03
   9.86166567e-05   3.88735553e-07   2.43887246e-04   2.58419899e-04
   1.80830996e-05   0.00000000e+00   2.05679519e-05   3.87782155e-05
   0.00000000e+00   1.40039254e-04   5.55428779e-04   5.88045851e-04
   9.75015655e-05   3.62073481e-04   1.77835143e-03   1.56755705e-03
   2.40324391e-04   2.34832806e-04   9.75155247e-04   9.24086393e-04
   3.39796546e-04   1.40105564e-04

In [107]:
rf_features = [1.82492709e-02,   1.74537425e-02,   1.46717675e-02,   3.84788633e-02,
   9.86989157e-02,   1.30347400e-01,   3.23392804e-01,   9.90428323e-02,
   1.14785667e-02,   8.10544292e-02,   3.45719827e-02,   2.36606906e-03,
   7.84745151e-04,   9.60120743e-03,   1.39975021e-03,   9.21511831e-04,
   1.73008250e-03,   1.55674870e-03,   0.00000000e+00,   1.81655396e-04,
   3.54847104e-04,   1.40601368e-03,   2.37748193e-03,   2.02960373e-03,
   2.36695462e-03,   9.02664948e-04,   3.38807814e-03,   4.97229944e-03,
   2.34640986e-03,   3.57723083e-04,   8.38641238e-03,   4.53421363e-03,
   5.35499003e-04,   3.47021655e-05,   1.47972941e-03,   1.28645941e-03,
   9.86166567e-05,   3.88735553e-07,   2.43887246e-04,   2.58419899e-04,
   1.80830996e-05,   0.00000000e+00,   2.05679519e-05,   3.87782155e-05,
   0.00000000e+00,   1.40039254e-04,   5.55428779e-04,   5.88045851e-04,
   9.75015655e-05,   3.62073481e-04,   1.77835143e-03,   1.56755705e-03,
   2.40324391e-04,   2.34832806e-04,   9.75155247e-04,   9.24086393e-04,
   3.39796546e-04,   1.40105564e-04,   7.98606545e-04,   9.11896575e-04,
   1.23048957e-04,   2.71997570e-04,   7.00104165e-04,   7.34255431e-04,
   5.77009103e-05,   8.02531911e-07,   4.06649906e-04,   3.23375196e-04,
   0.00000000e+00,   0.00000000e+00,   5.05690795e-05,   1.50105553e-05,
   0.00000000e+00,   1.52116423e-05,   1.39637142e-05,   0.00000000e+00,
   9.96699755e-03,   2.95160091e-03,   5.24023685e-03,   4.03526667e-03,
   1.47539668e-06,   4.30855091e-04,   4.69534389e-04,   0.00000000e+00,
   0.00000000e+00,   0.00000000e+00,   9.64745801e-06,   8.30008176e-07,
   0.00000000e+00,   0.00000000e+00,   1.15005826e-05,   2.79604082e-05,
   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
   0.00000000e+00,   0.00000000e+00,   4.43184077e-03,   4.44886129e-03,
   1.55869111e-02,   1.65998116e-02]
et_features = [ 2.26537697e-02,   1.17210253e-02,   2.65363349e-02,   2.62460598e-02,
   2.89372118e-02,   3.28322849e-02,   2.59841253e-01,   8.96426999e-02,
   1.49417978e-02,   4.03326139e-02,   2.33364193e-02,   8.62073394e-03,
   3.23405533e-03,   2.13468815e-02,   3.02456224e-03,   3.70954736e-03,
   6.23726317e-03,   6.27921710e-03,   0.00000000e+00,   5.93153196e-03,
   1.76242267e-03,   2.63917762e-03,   5.45408761e-03,   4.61073021e-03,
   6.07825572e-03,   2.62382964e-03,   1.04495303e-02,   1.15756468e-02,
   8.26350680e-03,   7.44298195e-04,   3.07020016e-02,   2.02991637e-02,
   1.78417319e-03,   1.05062668e-04,   4.06777613e-03,   5.28701261e-03,
   2.51712985e-04,   6.68914554e-06,   7.03567798e-04,   5.84312703e-04,
   2.70689613e-05,   0.00000000e+00,   1.49773793e-04,   1.73106025e-04,
   5.11591781e-05,   2.63470670e-04,   1.54710879e-03,   1.67767073e-03,
   4.11282403e-04,   1.71643156e-03,   6.22988711e-03,   5.04262730e-03,
   1.09761049e-03,   6.03067701e-04,   4.48072631e-03,   3.46599399e-03,
   8.98453243e-04,   3.19869571e-04,   2.19366035e-03,   2.15218184e-03,
   5.72982020e-04,   5.51784039e-04,   1.92488136e-03,   2.21718282e-03,
   4.91492457e-04,   2.73390630e-06,   8.56089238e-04,   1.05551912e-03,
   5.00314953e-06,   1.14809097e-05,   1.33167747e-04,   6.62598959e-05,
   6.83136027e-07,   1.24844991e-04,   1.47884438e-04,   0.00000000e+00,
   3.47877609e-02,   1.55069623e-02,   1.90652924e-02,   1.79868159e-02,
   7.42977565e-06,   6.39457290e-04,   8.39254924e-04,   1.15461018e-05,
   0.00000000e+00,   0.00000000e+00,   1.35240982e-04,   6.51151327e-05,
   0.00000000e+00,   0.00000000e+00,   5.36773320e-05,   7.16073710e-05,
   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
   0.00000000e+00,   0.00000000e+00,   1.39152706e-02,   1.44094611e-02,
   4.15858552e-02,   4.68578925e-02]
ada_features = [0.012,  0.068,  0.012,  0.056,  0.042,  0.026,  0.07,   0.032,  0.048,  0.414,
  0.084,  0.012,  0.002,  0.012,  0.,     0.002,  0.002,  0.,     0.002,  0.002,  0.,
  0.002,  0.004,  0.002,  0.002,  0.004,  0.004,  0.004,  0.002,  0.,     0.006,  0.,
  0.,     0.002,  0.,     0.002,  0.002,  0.,     0.,     0.,     0.002,  0.002,  0.,
  0.,     0.002,  0.,     0.,     0.,     0.,     0.002,  0.002,  0.,     0.002,  0.,
  0.002,  0.,     0.002,  0.,     0.,     0.,     0.,     0.002,  0.,     0.002,
  0.002,  0.002,  0.002,  0.,     0.002,  0.002,  0.,     0.002,  0.,     0.002,  0.,
  0.,     0.004,  0.,     0.004,  0.002,  0.002,  0.,     0.002,  0.002,  0.,     0.002,
  0.,     0.,     0.,     0.,     0.,     0.002,  0.,     0.,     0.,     0.002,  0.,
  0.002,  0.002,  0.,     0.002,  0.004]
gb_features = [ 7.57671642e-02,   1.48163194e-01,   4.47082982e-02,   1.09075088e-01,
   3.42867215e-02,   2.25623939e-02,   5.13468639e-02,   5.11040535e-02,
   3.81515794e-02,   7.47621805e-02,   4.43948266e-02,   8.78313703e-03,
   2.27734229e-03,   9.13799211e-03,   8.28588625e-03,   3.73680057e-03,
   6.87874312e-03,   6.12297172e-03,   7.69155569e-04,   1.25347947e-03,
   3.69469447e-03,   5.19070705e-03,   6.51370263e-03,   9.14179337e-03,
   1.27598339e-02,   1.11290228e-02,   1.16777995e-02,   1.24656159e-02,
   7.24857569e-03,   2.96912409e-03,   5.60896094e-03,   5.82008106e-03,
   3.63200683e-03,   7.12520655e-04,   3.30673905e-03,   3.59344970e-03,
   1.14723440e-03,   1.21411333e-03,   1.03173965e-03,   2.14151884e-03,
   8.97423155e-04,   0.00000000e+00,   4.10522400e-04,   7.71834157e-04,
   2.05440027e-03,   2.29704654e-03,   3.72390254e-03,   4.22902163e-03,
   2.20299034e-03,   3.94435273e-03,   6.15213761e-03,   7.48937439e-03,
   4.33250443e-03,   3.62182325e-03,   4.41673600e-03,   9.40241154e-03,
   4.26849086e-03,   2.22149090e-03,   3.24986887e-03,   5.75162983e-03,
   1.42055139e-03,   9.19988658e-04,   2.98240069e-03,   4.59090095e-03,
   9.00137254e-04,   3.08967426e-04,   7.07071888e-04,   1.43095262e-03,
   6.07396132e-04,   8.94314305e-04,   5.66438971e-04,   5.76871569e-04,
   2.05577907e-05,   7.24631891e-04,   9.04475346e-04,   0.00000000e+00,
   1.12645659e-02,   8.62379860e-03,   8.14908190e-03,   7.49066366e-03,
   1.35917086e-03,   1.58365916e-03,   2.48309900e-03,   1.33155538e-03,
   0.00000000e+00,   0.00000000e+00,   4.77185780e-04,   4.31097869e-04,
   0.00000000e+00,   0.00000000e+00,   5.80770445e-04,   4.33982962e-04,
   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
   0.00000000e+00,   0.00000000e+00,   2.50304915e-03,   4.40541814e-03,
   3.32934461e-03,   3.99283257e-03]

In [108]:
cols = train.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_features,
     'Extra Trees  feature importances': et_features,
      'AdaBoost feature importances': ada_features,
    'Gradient Boost feature importances': gb_features
    })

In [109]:
# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Random Forest feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Random Forest feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Extra Trees  feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Extra Trees  feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Extra Trees Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['AdaBoost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['AdaBoost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'AdaBoost Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Gradient Boost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Gradient Boost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [110]:
# Create the new column containing the average of values

feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
feature_dataframe.head(3)

,AdaBoost feature importances,Extra Trees feature importances,Gradient Boost feature importances,Random Forest feature importances,features,mean
0,0.012,0.022654,0.075767,0.018249,time_in_hospital,0.032168
1,0.068,0.011721,0.148163,0.017454,num_lab_procedures,0.061334
2,0.012,0.026536,0.044708,0.014672,num_procedures,0.024479


In [111]:
y = feature_dataframe['mean'].values
x = feature_dataframe['features'].values
data = [go.Bar(
            x= x,
             y= y,
            width = 0.5,
            marker=dict(
               color = feature_dataframe['mean'].values,
            colorscale='Portland',
            showscale=True,
            reversescale = False
            ),
            opacity=0.6
        )]

layout= go.Layout(
    autosize= True,
    title= 'Barplots of Mean Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-direct-labels')

Second-Level Predictions from the First-level Output

In [112]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
base_predictions_train.head()

,AdaBoost,ExtraTrees,GradientBoost,RandomForest
0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


Correlation Heatmap of the Second Level Training set

In [113]:
data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

In [114]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

Second level learning model via XGBoost

In [115]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict(x_test)

In [119]:
predictions

array([0, 0, 0, ..., 1, 0, 1])